In [1]:
import numpy as np
import torch
import torch.nn as nn
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio
import time
import torch.nn.utils as utils

pio.renderers.default = "plotly_mimetype+notebook"

In [123]:
class Network(nn.Module):

  def __init__(self, input, width, depth, output):
    super().__init__()

    # activation function
    activation = nn.Sigmoid

    # input layer
    self.input = nn.Sequential(*[nn.Linear(input, depth), activation()])

    # hidden layers
    self.hidden = nn.Sequential(*[nn.Sequential(*[nn.Linear(depth, depth), activation()]) for _ in range(width-2)])

    # output layer
    self.output = nn.Linear(depth, output)

    self.constants()

  def constants(self):
    self.L = 0.1 # m

    self.cp = 0.7 # J/gK
    self.p = 8000 # kg/m^3
    self.k = 15 # W/mK

    self.a = self.k/(self.p*self.cp) # m^2/s
    
    self.t_ref_t = 1473 # K
    self.t_ref_b = 1273 # K

    self.ht = 200
    self.hb = 100

    self.Bi_t =  self.ht*self.L/self.k
    self.Bi_b = self.hb*self.L/self.k   
    
    self.lossTotal = None
    
  def forward(self, x, t):
    inp = torch.cat((x, t), dim = 1)
    inp = self.input(inp)
    inp = self.hidden(inp)
    inp = self.output(inp)
    return inp
  
  def init_boundaries(self, N):

    # x and t boundary points for training x boundaries, x = -1,1, varying t values
    minusx_boundary = torch.tensor(-1.).repeat(N).view(-1, 1).requires_grad_(True).to(device)
    x_boundary = torch.tensor(1.).repeat(N).view(-1, 1).requires_grad_(True).to(device)
    t_boundary = 10*torch.rand(N).view(-1, 1).requires_grad_(True).to(device)

    # 0 time boundary for training
    x_IC = (-2*torch.rand(N)+1).view(-1, 1).requires_grad_(True).to(device) # X = -1,1 random values
    t_IC = torch.tensor(0.).repeat(N).view(-1, 1).requires_grad_(True).to(device) # T = 0
    
    self.BCpoints = [minusx_boundary, x_boundary, t_boundary]
    self.ICs = [x_IC, t_IC]
  
  def sample(self, N):
      
      # sample points for training
      x = (-2*torch.rand(N)+1).view(-1, 1).requires_grad_(True).to(device)
      t = (10*torch.rand(N)).view(-1, 1).requires_grad_(True).to(device)
  
      return x, t

  def derivative(self,f,x):
    return torch.autograd.grad(f, x, grad_outputs=torch.ones_like(f).to(device), create_graph=True)[0]
  
  def losses(self,N):

    self.init_boundaries(N)
    x_points, t_points = self.sample(N)

    # losses for x and t boundaries
    T = self.forward(self.BCpoints[0], self.BCpoints[2])
    T_x = self.derivative(T, self.BCpoints[0])
    
    minusx_loss = (T_x-self.Bi_b*T)**2
    
    T1 = self.forward(self.BCpoints[1], self.BCpoints[2])
    T_x1 = self.derivative(T1, self.BCpoints[1])
    
    x_loss = (T_x1+self.Bi_t*(T1-1))**2
  
    # loss for initial condition
    
    T_IC = self.forward(self.ICs[0], self.ICs[1])
    t0_loss = (T_IC-1)**2

    # loss for physics sample
    T_phy = self.forward(x_points, t_points)
    T_x_phy = self.derivative(T_phy, x_points)
    T_xx_phy = self.derivative(T_x_phy, x_points)
    T_t_phy = self.derivative(T_phy, t_points)
    
    phys_loss = (T_t_phy-4*T_xx_phy)**2
    
    loss = torch.mean(minusx_loss) + torch.mean(x_loss) + 100*torch.mean(t0_loss) + torch.mean(phys_loss)
    return None if torch.isnan(loss) else loss
  
  
  def trainAdam(self, epochs_max, lr, N):

    # initialise parameters and optimiser
    self.lossTotal = self.lossTotal or [1]
    loss_val = self.lossTotal[-1]
    epoch = len(self.lossTotal)-1
    optimizer = torch.optim.Adam(self.parameters(), lr = lr)
    
    epoch_times = []
    window_size = 10
    
    start = time.time()

    while epoch < epochs_max and loss_val > 1e-5:
      epoch += 1
      optimizer.zero_grad()
      # find loss
      loss = self.losses(N)
      if loss is None: break
      loss_val = loss.item()
      # store parameter and loss values
      self.lossTotal.append(loss.item())
      # backpropagation
      loss.backward()
      optimizer.step()
      
      # compute epoch time
      epoch_time = time.time() - start
      epoch_times.append(epoch_time)
      start = time.time()  # Reset the start time for the next epoch

      # Compute the moving average of epoch times
      if len(epoch_times) > window_size:
          epoch_times = epoch_times[-window_size:]
      avg_epoch_time = sum(epoch_times) / len(epoch_times)
    
      # print loss and parameter values
      print(f'Epoch: {epoch}  Loss: {loss_val:.8f} Time/Epoch: {avg_epoch_time:.4f}s', end='\r')
      if epoch in [1, 50, 100, 200, 500, 1000, 2000, 5000, 10000, 20000, 50000, 100000, 200000, 500000, 1000000]: self.plot()
      
  def trainLBFGS(self, epochs_max, N):
    
    max_norm = 1.0  # Maximum norm for clipping gradients
    
    self.lossTotal = self.lossTotal or [1]
    loss_val = self.lossTotal[-1]
    epoch = len(self.lossTotal)-1
    
    optimizer = torch.optim.LBFGS(self.parameters(), max_iter=1000, history_size=1000, tolerance_grad=1.0 * np.finfo(float).eps, tolerance_change=1.0 * np.finfo(float).eps, line_search_fn='strong_wolfe')
    
    epoch_times = []
    window_size = 10
    
    start = time.time()

    def closure():
      optimizer.zero_grad()
      loss = self.losses(N)
      if loss is None: return None
      loss.backward()
      return loss
  
    while epoch < epochs_max and loss_val > 1e-5:
      epoch += 1
      utils.clip_grad_norm_(self.parameters(), max_norm)
      loss = optimizer.step(closure)
      if loss is None: break
      loss_val = loss.item()
      self.lossTotal.append(loss_val)
      
      epoch_time = time.time() - start
      epoch_times.append(epoch_time)
      start = time.time()  # Reset the start time for the next epoch

      # Compute the moving average of epoch times
      if len(epoch_times) > window_size:
          epoch_times = epoch_times[-window_size:]
      avg_epoch_time = sum(epoch_times) / len(epoch_times)
      
      print(f'Epoch: {epoch}  Loss: {loss_val:.8f} Time/Epoch: {avg_epoch_time:.4f}s', end='\r')
      if epoch in [1, 50, 100, 200, 500, 1000, 2000, 5000, 10000, 20000, 50000, 100000, 200000, 500000, 1000000]: self.plot()
      
  def plot(self):
    X = torch.linspace(-1, 1, 100).to(device)
    T = torch.linspace(0, 10, 100).to(device)
    X_grid, T_grid = torch.meshgrid(X, T)

    Temp = (pinn.forward(X_grid.flatten().unsqueeze(1), T_grid.flatten().unsqueeze(1))).reshape(100, 100).cpu().detach().numpy() # Predicted temperature

    fig = go.Figure(data=[go.Surface(
    y=X.cpu().detach().numpy(),
    x=T.cpu().detach().numpy(),
    z=Temp,
    colorscale='magma'
    )])
    fig.update_layout(scene = dict(xaxis_title='X',yaxis_title='Time',zaxis_title='Temperature',xaxis_backgroundcolor="rgba(0, 0, 0, 0)",yaxis_backgroundcolor="rgba(0, 0, 0, 0)",zaxis_backgroundcolor="rgba(0, 0, 0, 0)"),
                      scene_camera = dict(eye=dict(x=-2, y=-2, z=2.5)),
                      plot_bgcolor='rgba(0, 0, 0, 0)', paper_bgcolor= 'rgba(0, 0, 0, 0)', font_color="white",
                      width=800,
                      height=800)
    fig.show()

In [130]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# nn seed
torch.manual_seed(123)

# define nn to train, two inputs for x and t
pinn = Network(2, 6, 20, 1).to(device)

# training things
epochs_max = 1000
lrate = 1e-4
N = 10000
iter = 500

In [ ]:
pinn.trainAdam(10000, 1e-2, N*10)

In [125]:
pinn.trainLBFGS(iter, N)
print()
print("Input weight gradients")
print(format(torch.mean(pinn.input[0].weight.grad).item(),'.3e'))
print("Hidden weight gradients")
print(format(np.mean([torch.mean(pinn.hidden[i][0].weight.grad).item() for i in range(4)]),'.3e'),sep='\n')
print("Output weight gradients")
print(format(torch.mean(pinn.output.weight.grad).item(),'.3e'))
print()
print("Input bias gradients")
print(format(torch.mean(pinn.input[0].bias.grad).item(),'.3e'))
print("Hidden bias gradients")
print(format(np.mean([torch.mean(pinn.hidden[i][0].bias.grad).item() for i in range(4)]),'.3e'),sep='\n')
print("Output bias gradients")
print(format(torch.mean(pinn.output.bias.grad).item(),'.3e'))
iter+=100

TypeError: float() argument must be a string or a real number, not 'NoneType'

In [122]:
X = torch.linspace(-1, 1, 100).to(device)
T = torch.linspace(0, 0, 100).to(device)

Temp = (pinn.forward(X.view(-1,1),T.view(-1,1))).cpu().detach().numpy() # Predicted temperature

fig = px.line(x=X.cpu(), y=Temp.reshape(100), log_y=False, range_y=[])
fig.update_traces(line_color='#a5843f')
fig.update_layout(plot_bgcolor='rgba(0, 0, 0, 0)', paper_bgcolor= 'rgba(0, 0, 0, 0)', font_color="white", title="Initial Condition", xaxis_title="X", yaxis_title="Temperature")
fig.show()

In [6]:
fig = px.line(x=range(len(pinn.lossTotal)), y=pinn.lossTotal, log_y=True, range_y=[])
fig.update_traces(line_color='#a5843f')
fig.update_layout(plot_bgcolor='rgba(0, 0, 0, 0)', paper_bgcolor= 'rgba(0, 0, 0, 0)', font_color="white", title="Loss vs Epochs", xaxis_title="Epochs", yaxis_title="Loss")
fig.show()